# This notebooks calls the edgefitting_2D.py routine and makes some pictures afterwards

In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import AdvancedBraggEdgeFitting_v2
from PIL import Image

import tifffile
from tifffile import TiffFile

from skimage import io

import os, fnmatch
from os import listdir

from TOF_routines import tof2l
from TOF_routines import find_nearest

from astropy.io import fits
import edgefitting_2D

import ipywe.fileselector

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
#TODO : remove the unused libs

%matplotlib notebook

### Get the path data 

In [2]:
fsel_data= ipywe.fileselector.FileSelectorPanel(instruction='select directory for data' , start_dir='.', type='directory')
fsel_data.show()
#ex. '/media/carminati_c/Data2/LSP_Manuel/sample_binned_spotCleaned/'

### Get the path OB 

In [3]:
fsel_ob= ipywe.fileselector.FileSelectorPanel(instruction='select directory for open beam' , start_dir='.', type='directory')
fsel_ob.show()
#ex. '/media/carminati_c/Data2/LSP_Manuel/OB_binned_spotCleaned/'

### Get the spectrum txt file

In [4]:
fsel_spectrum = ipywe.fileselector.FileSelectorPanel(instruction='select Spectra.txt' , start_dir='.', type='file')
fsel_spectrum.show()
#ex. '/media/ws_niag/10_people/Morgano/RADEN_data_analysis/TEST6_000_Spectra.txt

### Get the mask file 

In [5]:
fsel_mask = ipywe.fileselector.FileSelectorPanel(instruction='select mask file' , start_dir='.', type='file')
fsel_mask.show()
#ex. /home/carminati_c/git/ToFImaging/scripts/mymask.tif

### Define the parameters needed for the edgefitting_2D function 

In [6]:
pathdata = fsel_data.selected
pathob = fsel_ob.selected
filemask = fsel_mask.selected
files_sample = (sorted(fnmatch.filter(listdir(pathdata),'*.tif')))
files_ob = (sorted(fnmatch.filter(listdir(pathob),'*.tif')))
spectrum = fsel_spectrum.selected

In [7]:
# These numbers come from TOF_Calibration
t0=-0.0002618673892937752 
L = 18.5
cal_parameters = np.array([t0, L])
#other initialization parameters
binning = 18
lambda_range = np.array([3.7, 4.4])
est_parameters = np.array([0.1, 0.1, 4.05]) # sigma, alpha, edge_position

In [8]:
res = edgefitting_2D.image_edge_fitting(pathdata, pathob, filemask, spectrum, cal_parameters, binning, lambda_range, est_parameters)

/home/carminati_c/anaconda3/lib/python3.7/site-packages/lmfit/minimizer.py:767: RuntimeWarning: invalid value encountered in sqrt
  par.stderr = sqrt(self.result.covar[ivar, ivar])
/home/carminati_c/anaconda3/lib/python3.7/site-packages/lmfit/minimizer.py:774: RuntimeWarning: invalid value encountered in sqrt
  (par.stderr * sqrt(self.result.covar[jvar, jvar])))
/home/carminati_c/git/ToFImaging/scripts/AdvancedBraggEdgeFitting_v2.py:56: RuntimeWarning: invalid value encountered in multiply
  edge = 0.5*(term3(t,t0,sigma) - term4(t,t0,alpha,sigma)* term5(t,t0,alpha,sigma))
/home/carminati_c/anaconda3/lib/python3.7/site-packages/scipy/optimize/_lsq/common.py:501: RuntimeWarning: invalid value encountered in less
  mask = (g < 0) & np.isfinite(ub)
/home/carminati_c/anaconda3/lib/python3.7/site-packages/scipy/optimize/_lsq/common.py:505: RuntimeWarning: invalid value encountered in greater
  mask = (g > 0) & np.isfinite(lb)


Unexpected error at : 34 362
Unexpected error at : 34 368
Unexpected error at : 35 303
Unexpected error at : 35 326
Unexpected error at : 35 340
Unexpected error at : 35 341
Unexpected error at : 35 373
Unexpected error at : 36 301
Unexpected error at : 36 361
Unexpected error at : 37 291
Unexpected error at : 37 303
Unexpected error at : 37 317
Unexpected error at : 38 214
Unexpected error at : 38 231
Unexpected error at : 38 240
Unexpected error at : 38 257
Unexpected error at : 38 266
Unexpected error at : 38 280
Unexpected error at : 38 296
Unexpected error at : 38 310
Unexpected error at : 38 319
Unexpected error at : 38 335
Unexpected error at : 39 225
Unexpected error at : 39 257
Unexpected error at : 39 261
Unexpected error at : 39 316
Unexpected error at : 39 331
Unexpected error at : 40 211
Unexpected error at : 40 271
Unexpected error at : 40 332
Unexpected error at : 40 346
Unexpected error at : 40 365
Unexpected error at : 40 369
Unexpected error at : 40 378
Unexpected err

Unexpected error at : 62 338
Unexpected error at : 63 29
Unexpected error at : 63 110
Unexpected error at : 63 198
Unexpected error at : 63 299
Unexpected error at : 63 308
Unexpected error at : 63 312
Unexpected error at : 63 324
Unexpected error at : 63 331
Unexpected error at : 63 370
Unexpected error at : 63 379
Unexpected error at : 64 76
Unexpected error at : 64 78
Unexpected error at : 64 87
Unexpected error at : 64 117
Unexpected error at : 64 160
Unexpected error at : 64 179
Unexpected error at : 64 192
Unexpected error at : 64 220
Unexpected error at : 64 238
Unexpected error at : 64 243
Unexpected error at : 64 251
Unexpected error at : 64 253
Unexpected error at : 64 271
Unexpected error at : 64 274
Unexpected error at : 65 88
Unexpected error at : 65 113
Unexpected error at : 65 148
Unexpected error at : 65 168
Unexpected error at : 65 202
Unexpected error at : 65 203
Unexpected error at : 65 259
Unexpected error at : 65 280
Unexpected error at : 65 327
Unexpected error at

Unexpected error at : 84 230
Unexpected error at : 84 241
Unexpected error at : 84 254
Unexpected error at : 84 256
Unexpected error at : 84 332
Unexpected error at : 84 346
Unexpected error at : 84 350
Unexpected error at : 84 358
Unexpected error at : 85 60
Unexpected error at : 85 96
Unexpected error at : 85 123
Unexpected error at : 85 140
Unexpected error at : 85 189
Unexpected error at : 85 268
Unexpected error at : 85 269
Unexpected error at : 85 286
Unexpected error at : 85 316
Unexpected error at : 85 351
Unexpected error at : 85 357
Unexpected error at : 85 358
Unexpected error at : 85 367
Unexpected error at : 86 67
Unexpected error at : 86 105
Unexpected error at : 86 130
Unexpected error at : 86 178
Unexpected error at : 86 197
Unexpected error at : 86 211
Unexpected error at : 86 224
Unexpected error at : 86 265
Unexpected error at : 87 24
Unexpected error at : 87 39
Unexpected error at : 87 53
Unexpected error at : 87 99
Unexpected error at : 87 104
Unexpected error at :

Unexpected error at : 106 83
Unexpected error at : 106 90
Unexpected error at : 106 92
Unexpected error at : 106 100
Unexpected error at : 106 145
Unexpected error at : 106 155
Unexpected error at : 106 165
Unexpected error at : 106 175
Unexpected error at : 106 186
Unexpected error at : 106 215
Unexpected error at : 106 216
Unexpected error at : 106 220
Unexpected error at : 106 221
Unexpected error at : 106 309
Unexpected error at : 106 352
Unexpected error at : 106 372
Unexpected error at : 107 22
Unexpected error at : 107 30
Unexpected error at : 107 55
Unexpected error at : 107 72
Unexpected error at : 107 101
Unexpected error at : 107 179
Unexpected error at : 107 260
Unexpected error at : 107 261
Unexpected error at : 107 282
Unexpected error at : 107 321
Unexpected error at : 108 140
Unexpected error at : 108 173
Unexpected error at : 108 174
Unexpected error at : 108 178
Unexpected error at : 108 211
Unexpected error at : 108 271
Unexpected error at : 108 316
Unexpected error 

...

Unexpected error at : 238 268
Unexpected error at : 238 314
Unexpected error at : 238 357
Unexpected error at : 239 28
Unexpected error at : 239 73
Unexpected error at : 239 101
Unexpected error at : 239 102
Unexpected error at : 239 124
Unexpected error at : 239 126
Unexpected error at : 239 127
Unexpected error at : 239 147
Unexpected error at : 239 233
Unexpected error at : 239 247
Unexpected error at : 239 267
Unexpected error at : 239 276
Unexpected error at : 239 305
Unexpected error at : 240 32
Unexpected error at : 240 37
Unexpected error at : 240 40
Unexpected error at : 240 41
Unexpected error at : 240 64
Unexpected error at : 240 93
Unexpected error at : 240 102
Unexpected error at : 240 137
Unexpected error at : 240 163
Unexpected error at : 240 164
Unexpected error at : 240 234
Unexpected error at : 240 246
Unexpected error at : 240 302
Unexpected error at : 240 315
Unexpected error at : 240 361
Unexpected error at : 241 38
Unexpected error at : 241 42
Unexpected error at 

Unexpected error at : 259 234
Unexpected error at : 259 246
Unexpected error at : 259 247
Unexpected error at : 259 255
Unexpected error at : 259 259
Unexpected error at : 259 344
Unexpected error at : 259 375
Unexpected error at : 260 33
Unexpected error at : 260 68
Unexpected error at : 260 121
Unexpected error at : 260 137
Unexpected error at : 260 161
Unexpected error at : 260 181
Unexpected error at : 260 198
Unexpected error at : 260 210
Unexpected error at : 260 245
Unexpected error at : 260 257
Unexpected error at : 260 270
Unexpected error at : 260 321
Unexpected error at : 260 341
Unexpected error at : 261 26
Unexpected error at : 261 39
Unexpected error at : 261 53
Unexpected error at : 261 143
Unexpected error at : 261 164
Unexpected error at : 261 175
Unexpected error at : 261 178
Unexpected error at : 261 217
Unexpected error at : 261 307
Unexpected error at : 261 311
Unexpected error at : 261 329
Unexpected error at : 261 340
Unexpected error at : 261 349
Unexpected erro

Unexpected error at : 278 92
Unexpected error at : 278 136
Unexpected error at : 278 177
Unexpected error at : 278 205
Unexpected error at : 278 213
Unexpected error at : 278 278
Unexpected error at : 278 319
Unexpected error at : 278 347
Unexpected error at : 278 349
Unexpected error at : 279 149
Unexpected error at : 279 155
Unexpected error at : 279 234
Unexpected error at : 279 243
Unexpected error at : 279 333
Unexpected error at : 280 57
Unexpected error at : 280 99
Unexpected error at : 280 135
Unexpected error at : 280 139
Unexpected error at : 280 140
Unexpected error at : 280 148
Unexpected error at : 280 152
Unexpected error at : 280 166
Unexpected error at : 280 201
Unexpected error at : 280 211
Unexpected error at : 280 218
Unexpected error at : 280 221
Unexpected error at : 280 226
Unexpected error at : 280 251
Unexpected error at : 280 318
Unexpected error at : 280 321
Unexpected error at : 280 364
Unexpected error at : 280 372
Unexpected error at : 281 47
Unexpected err

Unexpected error at : 299 230
Unexpected error at : 299 250
Unexpected error at : 299 260
Unexpected error at : 299 267
Unexpected error at : 299 296
Unexpected error at : 299 305
Unexpected error at : 299 309
Unexpected error at : 299 314
Unexpected error at : 299 319


NameError: name 'edge_height' is not defined

In [ ]:
edge_pos = np.

### Example how to compute initial parameters: 

In [ ]:
files_ob = (sorted(fnmatch.filter(listdir(pathob),'*.tif')))
# print(files_ob)
ob = io.imread(pathob+'/'+files_ob[0])
np.array([np.shape(ob)[0], np.shape(ob)[1], len(files_ob)])

In [ ]:
#load spectrum, bin, and convert to lambda
spec = np.loadtxt(spectrum, usecols=0)
spectrum_binned = spec[0::18]
print(len(spectrum_binned))
mylambda_bin = tof2l(spectrum_binned,t0,L)

In [ ]:
#load ob and sample images
print(pathob+'/'+files_ob[0])

ob = io.imread(pathob+'/'+files_ob[0])
np.shape(ob)

ob_image = np.zeros([300,400,155])
sample_image = np.zeros([300,400,155])
np.shape(ob_image)

for i in range(0,155):
    ob_image[:,:,i] = io.imread(pathob+'/'+files_ob[i])
    sample_image[:,:,i] = io.imread(pathdata+'/'+files_sample[i])

In [ ]:
# compute transmission
trans_image = np.zeros([300,400,155])
for i in range(0,155):
    dose = np.median(ob_image[10:20,10:20,i])/np.median(sample_image[10:20,10:20,i])
    trans_image[:,:,i] = sample_image[:,:,i]/ob_image[:,:,i]*dose

In [ ]:
plt.figure()
plt.imshow(trans_image[:,:,70],cmap='gray')
plt.axis('off')
# plt.savefig('transmission_lsp.pdf')

In [ ]:
#compute spectrum over a large region to compute test initial parameters
sp=np.zeros(156)
for i in range(0,155):
    sp[i] = np.median(trans_image[37,254,i])

In [ ]:
plt.figure()
plt.plot(mylambda_bin, sp)
plt.title('total spectrum')

In [ ]:
%%time
myrange =[]
myrange.append(find_nearest(mylambda_bin, 3.7))
myrange.append(find_nearest(mylambda_bin, 4.4))
print(myrange)

plt.figure()
plt.plot(mylambda_bin[myrange[0]:myrange[1]], sp[myrange[0]:myrange[1]])

est_sigma = 0.1
est_alpha = 0.1
est_pos = find_nearest(mylambda_bin, 4.05)
print(est_pos)
ris = AdvancedBraggEdgeFitting_v2.AdvancedBraggEdgeFitting(sp, myrange, mylambda_bin, est_pos, est_sigma, est_alpha, True, False, False, True)
# t_edg = ris['t0']
# print(t_edg)

### This is an example how to create a mask

In [ ]:
# This in an example how to create the mask
ob = io.imread(pathob+'/'+files_ob[0])
np.shape(ob)

ob_image = np.zeros([300,400,155])
sample_image = np.zeros([300,400,155])
np.shape(ob_image)

for i in range(0,155):
    ob_image[:,:,i] = io.imread(pathob+'/'+files_ob[i])
    sample_image[:,:,i] = io.imread(pathdata+'/'+files_sample[i])


trans_image = np.zeros([300,400,155])
for i in range(0,155):
    dose = np.median(ob_image[10:20,10:20,i])/np.median(sample_image[10:20,10:20,i])
    trans_image[:,:,i] = sample_image[:,:,i]/ob_image[:,:,i]*dose

mymask = trans_image[:,:,70]<0.5

mymask[:28,:] = False
mymask[:,380:] = False

plt.figure()
plt.imshow(mymask)
plt.axis('off')
plt.axis('tight')
# io.imsave('mymask.tif', mymask+0)

### Example on data plotting after 2D edge fitting 

In [15]:
test = np.load('/edge_saved/edge_position.npy')
plt.figure()
plt.imshow(test,vmin=4.10, vmax=4.15)
plt.colorbar()

FileNotFoundError: [Errno 2] No such file or directory: '/edge_saved/edge_position.npy'

In [ ]:
# Check the number of points where the fitting fails
sum(sum(n < 0 for n in EP))

In [ ]:
# Re-run the fitting in those points

#     t0:                 4.07855978 +/- 15750.9720 (386189.56%) (init = 4.076283)
#     alpha:              8.8506e-04 +/- 26892.9752 (3038556174.03%) (init = 0.007669497)
#     sigma:              2.0660e-04 +/- 62396.2248 (30201102063.85%) (init = 0.0001408172)
#     a1:                 1.94456283 +/- 1.28224664 (65.94%) (init = 0.9776328)
#     a2:                -0.03410872 +/- 0.30144455 (883.78%) (init = 0.1905023)
#     a5:                -4.32338756 +/- 1.99813278 (46.22%) (init = -3.757007)
#     a6:                 1.29564172 +/- 0.49946988 (38.55%) (init = 1.176675)
        
est_alpha = 0.1
est_sigma = 0.1
a1 = 1.94456283
a2 = -0.03410872
a5 = -4.32338756
a6 = 1.29564172
est_pos = find_nearest(mylambda_bin[myrange[0]:myrange[1]], 4.07)

missing_edges = np.zeros(np.shape(mymask))
missing_heigths = np.zeros(np.shape(mymask))
print(est_pos)

print(myrange)
print(small_range)


# Re-run the edge fitting where the first set of parameters failed
for i in range(0, np.shape(mymask)[0]):
    
    for j in range(0, np.shape(mymask)[1]):
        
        if (EP[i,j]==-2):
            # extract the signal
            mysignal = np.zeros(myrange[1]-myrange[0])
            
    
            for ind in range(myrange[0],myrange[1]):
                mysignal[ind-myrange[0]] = np.median(trans_image[i,j,ind])
            
            try:
                edge_fit = AdvancedBraggEdgeFitting_v2.AdvancedBraggEdgeFitting(mysignal, small_range, small_lambda, est_pos, est_sigma, est_alpha, False, False, False, True)
                missing_edges[i,j] = edge_fit['t0']
                missing_heigths[i,j] = edge_fit['height']
            except:
                print("Unexpected error at :", i, j)
                missing_edges[i,j]= -2.0
                missing_heigths[i,j] = -2.0

                

In [ ]:
plt.figure()
plt.imshow(missing_edges)

In [ ]:
# Run again for the further missing edges 
est_alpha = 0.09
est_sigma = 0.09
a1 = 1.94456283
a2 = -0.03410872
a5 = -4.32338756
a6 = 1.29564172
est_pos = find_nearest(mylambda_bin[myrange[0]:myrange[1]], 4.05)


for i in range(0, np.shape(mymask)[0]):
    
    for j in range(0, np.shape(mymask)[1]):
        
        if (missing_edges[i,j]==-2):
            # extract the signal
            mysignal = np.zeros(myrange[1]-myrange[0])
            
    
            for ind in range(myrange[0],myrange[1]):
                mysignal[ind-myrange[0]] = np.median(trans_image[i,j,ind])
            
            try:
                edge_fit = AdvancedBraggEdgeFitting_v2.AdvancedBraggEdgeFitting(mysignal, small_range, small_lambda, est_pos, est_sigma, est_alpha, False, False, False, True)
                missing_edges[i,j] = edge_fit['t0']
                missing_heigths[i,j] = edge_fit['height']

            except:
                print("Unexpected error at :", i, j)
                missing_edges[i,j]= -2.0
                missing_heigths[i,j] = -2.0

In [ ]:
updated_EP = np.zeros(np.shape(mymask))

for i in range(0, np.shape(mymask)[0]):
    for j in range(0, np.shape(mymask)[1]):
        if (EP[i,j]!=-2):
            updated_EP[i,j]= EP[i,j]
        else:
            updated_EP[i,j]= missing_edges[i,j]
# updated_EP = EP[EP!=-2]
# updated_EP = missing_edges[EP==-2]

In [ ]:
plt.figure()
plt.imshow(np.rot90(updated_EP,3),vmin=4.05, vmax=4.09)
plt.axis('off')
plt.colorbar()
plt.savefig('LSP_model.pdf')

In [ ]:
#saving some results
np.save('edge_positions_all.npy', updated_EP)
fits.writeto('edge_pos_all.fits',updated_EP)